In [2]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/var/folders/t3/16zv_09j0zzb0hsbdwbwlz1h0000gn/T/ipykernel_78105/578044757.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# Read data
dataset = pd.read_csv(r"data/train.csv")

# Drop ID column
dataset = dataset.drop(dataset.columns[0], axis=1)
dataset.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# Extract features and pred
X = dataset.drop(["Survived", "Name"], axis=1)
y = dataset["Survived"]
X.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,female,35.0,1,0,113803,53.1000,C123,S
4,3,male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
# Need to encode text into numerical representation
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

X["Sex"] = label_encoder.fit_transform(X["Sex"])
X["Embarked"] = label_encoder.fit_transform(X["Embarked"])
X["Ticket"] = label_encoder.fit_transform(X["Ticket"])
X["Cabin"] = label_encoder.fit_transform(X["Cabin"])

# Clean up NaNs
nan_indices = X[X.isnull().any(axis=1)].index
X = X.drop(index=nan_indices)
y = y.drop(index=nan_indices)

print(X.shape)
print(y.shape)

# X.head()

(714, 9)
(714,)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=2)

gnb = GaussianNB()

y_pred = gnb.fit(X_train, y_train).predict(X_test)

In [8]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy {accuracy}")

Accuracy 0.786


In [25]:
test_df = pd.read_csv(r'data/test.csv')
test_df.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [26]:
from sklearn.impute import SimpleImputer

pass_ids = test_df["PassengerId"].copy()
test_df.drop(columns=["PassengerId", "Name"], inplace=True)

test_df["Sex"] = label_encoder.fit_transform(test_df["Sex"])
test_df["Embarked"] = label_encoder.fit_transform(test_df["Embarked"])
test_df["Ticket"] = label_encoder.fit_transform(test_df["Ticket"])
test_df["Cabin"] = label_encoder.fit_transform(test_df["Cabin"])


imputer = SimpleImputer(strategy='mean')
test_df = pd.DataFrame(imputer.fit_transform(test_df), columns=test_df.columns)


predictions = gnb.predict(test_df)
output_df = pd.DataFrame({"PassengerID": pass_ids, 'Survived': predictions})
output_df.to_csv('pred.csv', index=False)
print("done! csv saved.")


done! csv saved.
